这一小节我们演示如何使用千帆 SDK 快速调用大模型服务，如何使用 Prompt 模版以及使用 LangChain 提供的 Prompt 组件。

运行千帆的大模型服务首先要设置 AK 和 SK

In [27]:
import os
import qianfan

os.environ['QIANFAN_AK'] = "your_ak"
os.environ['QIANFAN_SK'] = "your_sk"

下面我们创建一个工具函数来方便的进行大模型调用。

In [36]:
def get_completion(prompt, model="ERNIE-Bot-4"):
    
    chat_comp = qianfan.ChatCompletion()

    # 指定特定模型
    resp = chat_comp.do(model=model, messages=[{
        "role": "user",
        "content": prompt
    }])

    return resp["body"]["result"]

In [37]:
get_completion("你好")

'你好，我是百度公司开发的人工智能语言模型，文心一言。我可以协助你完成范围广泛的任务并提供有关各种主题的信息，比如回答问题，提供定义和解释及建议。如果你有任何问题，请随时向我提问。'

当我们需要完成一个较为复杂的任务，通常会引入 Prompt 模版。

In [5]:
"""完成一个复杂的任务"""
personal_skills = """
我拥有扎实的计算机科学和人工智能背景，以及多年的自然语言处理经验。\
在之前的工作中，我参与了多个项目，包括机器翻译、情感分析、文本生成等，\
这些项目锻炼了我的算法设计和工程实施能力。我也深入研究了深度学习技术，\
并且在实际应用中获得了显著的成果。
"""

style = """使用尊敬的口吻并且要符合写信的规范格式"""

In [6]:
prompt = f"""请根据引号中的个人技能“{personal_skills}”写一封发给文心一言团队的求职信，要求为：{style}
"""
print(prompt)

请根据引号中的个人技能“
我拥有扎实的计算机科学和人工智能背景，以及多年的自然语言处理经验。在之前的工作中，我参与了多个项目，包括机器翻译、情感分析、文本生成等，这些项目锻炼了我的算法设计和工程实施能力。我也深入研究了深度学习技术，并且在实际应用中获得了显著的成果。
”写一封发给文心一言团队的求职信，要求为：使用尊敬的口吻并且要符合写信的规范格式



In [7]:
response = get_completion(prompt)

In [8]:
response

'尊敬的文心一言团队：\n\n您好！\n\n我在得知贵团队正在寻找具有计算机科学和人工智能背景的人才后，不禁想起了我自身的经历和专业技能，因此我怀着热切的心情写下这封求职信，希望能加入贵团队，共同追求卓越。\n\n我拥有扎实的计算机科学和人工智能背景，以及多年的自然语言处理经验。在之前的工作中，我参与了多个项目，包括机器翻译、情感分析、文本生成等，这些项目锻炼了我的算法设计和工程实施能力。我也深入研究了深度学习技术，并且在实际应用中获得了显著的成果。我相信，这些经验和技能使我能够迅速融入贵团队的工作环境，并贡献出我自己的力量。\n\n我对贵团队的研究方向和目标充满了敬意和热情。我一直关注着贵团队的发展，对贵团队在人工智能领域取得的成就深感钦佩。我希望能有机会与你们一同工作，共同推动人工智能技术的发展。\n\n如果我有幸成为贵团队的一员，我将尽我最大的努力，用我的专业知识和经验为团队的发展做出贡献。我相信，我能够迅速适应贵团队的工作环境，并与团队成员们共同推动项目的进展。\n\n感谢您抽出宝贵的时间阅读我的求职信。我期待能有机会与您进一步交流，详细介绍我的经历和能力。希望能有机会加入贵团队，共同为人工智能技术的发展做出贡献。\n\n祝您工作顺利，身体健康！\n\nXXX'

在 LangChain 中封装好了模型和 Prompt 组件可以减少我们的开发工作。

In [9]:
from langchain.chat_models import QianfanChatEndpoint
qianfan_chat = QianfanChatEndpoint(model="ERNIE-Bot-4")

In [11]:
from langchain.prompts import ChatPromptTemplate
template_string = """请根据引号中的个人技能“{personal_skills}”写一封发给文心一言团队的求职信，要求为：{style}
"""
prompt_template = ChatPromptTemplate.from_template(template_string)

Prompt 组件可以为我们定义一个字符串模版，以及模版中可配置的参数。

In [12]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['personal_skills', 'style'], template='请根据引号中的个人技能“{personal_skills}”写一封发给文心一言团队的求职信，要求为：{style}\n')

In [13]:
prompt_template.messages[0].prompt.input_variables

['personal_skills', 'style']

In [15]:
"""完成一个复杂的任务"""
personal_skills = """
我拥有扎实的计算机科学和人工智能背景，以及多年的自然语言处理经验。\
在之前的工作中，我参与了多个项目，包括机器翻译、情感分析、文本生成等，\
这些项目锻炼了我的算法设计和工程实施能力。我也深入研究了深度学习技术，\
并且在实际应用中获得了显著的成果。
"""

style = """使用尊敬的口吻并且要符合写信的规范格式"""

In [16]:
customer_messages = prompt_template.format_messages(
    style = style,
    personal_skills = personal_skills
)

In [17]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain.schema.messages.HumanMessage'>


In [18]:
print(customer_messages[0])

content='请根据引号中的个人技能“\n我拥有扎实的计算机科学和人工智能背景，以及多年的自然语言处理经验。在之前的工作中，我参与了多个项目，包括机器翻译、情感分析、文本生成等，这些项目锻炼了我的算法设计和工程实施能力。我也深入研究了深度学习技术，并且在实际应用中获得了显著的成果。\n”写一封发给文心一言团队的求职信，要求为：使用尊敬的口吻并且要符合写信的规范格式\n'


In [19]:
customer_response = qianfan_chat(customer_messages)

In [20]:
print(customer_response.content)

尊敬的文心一言团队：

您好！

我在人工智能领域拥有扎实的计算机科学背景，以及多年的自然语言处理经验。我很高兴有机会向您的团队提交我的求职申请。我对贵团队在人工智能领域的卓越贡献深感敬佩，并希望能有机会加入您的团队。

在我的职业生涯中，我参与了多个与自然语言处理相关的项目，包括机器翻译、情感分析和文本生成等。这些项目不仅锻炼了我的算法设计和工程实施能力，也使我对深度学习技术有了深入的研究。我在实际应用中获得了显著的成果，并相信这些经验将有助于我在文心一言团队做出贡献。

我热爱人工智能研究，并始终关注最新的技术和趋势。我相信我的背景和经验将使我能够迅速融入您的团队，并为实现共同的目标做出贡献。

感谢您抽出宝贵的时间阅读我的求职信。我期待有机会与您进一步讨论我的资历和如何在您的团队中发挥我的潜力。如有需要，我随时愿意提供更多信息或安排面试。

再次感谢您的考虑，期待您的回复。

祝好！

[您的姓名]


通过更换模版中的 style 参数，可以让大模型返回一个不同的答案。

In [21]:
"""完成一个复杂的任务"""
personal_skills = """
我拥有扎实的计算机科学和人工智能背景，以及多年的自然语言处理经验。\
在之前的工作中，我参与了多个项目，包括机器翻译、情感分析、文本生成等，\
这些项目锻炼了我的算法设计和工程实施能力。我也深入研究了深度学习技术，\
并且在实际应用中获得了显著的成果。
"""

style = """use English"""

qianfan_chat = QianfanChatEndpoint(model="ERNIE-Bot-4")

customer_messages = prompt_template.format_messages(
    style = style,
    personal_skills = personal_skills
)

customer_response = qianfan_chat(customer_messages)
print(customer_response.content)

Subject: Application for Position in Wenxin AI Team

Dear Hiring Managers,

I am writing to express my strong interest in the position offered at Wenxin AI Team. With my solid background in computer science and artificial intelligence, as well as years of experience in natural language processing, I believe I would be a great fit for the team.

In my previous work, I have been involved in multiple projects related to machine translation, sentiment analysis, text generation, etc. These projects have not only honed my skills in algorithm design and engineering implementation but also provided me with a deep understanding of the practical applications of AI technology. In addition, I have conducted in-depth research on deep learning technology and achieved remarkable results in practical applications.

I am confident that my expertise and experience would enable me to make valuable contributions to the Wenxin AI Team. I am excited about the opportunity to collaborate with the team and exp

可以将模型替换为千帆平台当前内置的其他模型服务

In [38]:
"""完成一个复杂的任务"""
personal_skills = """
我拥有扎实的计算机科学和人工智能背景，以及多年的自然语言处理经验。\
在之前的工作中，我参与了多个项目，包括机器翻译、情感分析、文本生成等，\
这些项目锻炼了我的算法设计和工程实施能力。我也深入研究了深度学习技术，\
并且在实际应用中获得了显著的成果。
"""

style = """使用尊敬的口吻并且要符合写信的规范格式"""

qianfan_chat = QianfanChatEndpoint(model="Qianfan-Chinese-Llama-2-7B")

customer_messages = prompt_template.format_messages(
    style = style,
    personal_skills = personal_skills
)

customer_response = qianfan_chat(customer_messages)
print(customer_response.content)

[WARNING] [10-26 13:51:19] logging.py:99 [t:8539758080]: This key `top_p` does not seem to be a parameter that the model `Qianfan-Chinese-Llama-2-7B` will accept
[WARNING] [10-26 13:51:19] logging.py:99 [t:8539758080]: This key `temperature` does not seem to be a parameter that the model `Qianfan-Chinese-Llama-2-7B` will accept
[WARNING] [10-26 13:51:19] logging.py:99 [t:8539758080]: This key `penalty_score` does not seem to be a parameter that the model `Qianfan-Chinese-Llama-2-7B` will accept


尊敬的文心一言团队：
我非常荣幸能够向您表达我的求职意向。我拥有扎实的计算机科学和人工智能背景，以及多年的自然语言处理经验。在之前的工作中，我参与了多个项目，包括机器翻译、情感分析、文本生成等，这些项目锻炼了我的算法设计和工程实施能力。
我深入研究了深度学习技术，并且在实际应用中获得了显著的成果。我相信这些技能和经验可以为文心一言团队带来价值，并能够与团队一起成长。
感谢您抽出时间阅读我的求职信。我期待着能够有机会加入您的团队，为公司的发展做出贡献。
此致
敬礼
XXX


也可以通过设置 endpoint 参数使用用户在千帆平台上自定义发布的大模型服务

In [41]:
"""完成一个复杂的任务"""
personal_skills = """
我拥有扎实的计算机科学和人工智能背景，以及多年的自然语言处理经验。\
在之前的工作中，我参与了多个项目，包括机器翻译、情感分析、文本生成等，\
这些项目锻炼了我的算法设计和工程实施能力。我也深入研究了深度学习技术，\
并且在实际应用中获得了显著的成果。
"""

style = """使用尊敬的口吻并且要符合写信的规范格式"""

qianfan_chat = QianfanChatEndpoint(endpoint="rr98bhif_agent_v4_8k")

customer_messages = prompt_template.format_messages(
    style = style,
    personal_skills = personal_skills
)

customer_response = qianfan_chat(customer_messages)
print(customer_response.content)

尊敬的文心一言团队，

我写信是希望能够应聘您在引号中所提到的个人技能。我拥有扎实的计算机科学和人工智能背景，以及多年的自然语言处理经验。在之前的工作中，我参与了多个项目，包括机器翻译、情感分析、文本生成等，这些项目锻炼了我的算法设计和工程实施能力。我也深入研究了深度学习技术，并且在实际应用中获得了显著的成果。

我相信我具备与您的团队合作的能力和背景，我将以积极的态度和团队合作精神加入文心一言团队，为推进自然语言处理技术而努力。

再次感谢您考虑我的申请。我期待着能够有机会和您进一步讨论我的能力和如何为文心一言团队做出贡献。

尊敬的文心一言团队，

我写信是希望能够应聘您在引号中所提到的个人技能。我拥有扎实的计算机科学和人工智能背景，以及多年的自然语言处理经验。在之前的工作中，我参与了多个项目，包括机器翻译、情感分析、文本生成等，这些项目锻炼了我的算法设计和工程实施能力。我也深入研究了深度学习技术，并且在实际应用中获得了显著的成果。

我相信我具备与您的团队合作的能力和背景，我将以积极的态度和团队合作精神加入文心一言团队，为推进自然语言处理技术而努力。

再次感谢您考虑我的申请。我期待着能够有机会和您进一步讨论我的能力和如何为文心一言团队做出贡献。
